### Cài đặt thư viện

In [1]:
! pip install fiftyone==0.21.4
! pip install torch torchvision torchaudio
! pip install ipywidgets
! pip install -U pinecone-client
! pip install httpcore==0.17.3
! pip install httpx==0.24.1
! pip install h11==0.14.0

Load dữ liệu keyframe từ thư mục chứa keyframe. Mỗi ảnh và thông tin đi kèm sau này sẽ được lưu trữ trong một Sample. Tất cả các Sample được lưu trong Dataset.

In [2]:
import fiftyone as fo
import fiftyone.brain as fob
import numpy as np
from glob import glob
import json
import os

### Tải dữ liệu lên
Load dữ liệu keyframe từ thư mục chứa keyframe. Mỗi ảnh và thông tin đi kèm sau này sẽ được lưu trữ trong một `Sample`. Tất cả các `Sample` được lưu trong `Dataset`.

In [3]:
# load dataset
dataset = fo.Dataset.from_images_dir('/Users/duyennguyen/Downloads/AIC/Batch_01/Keyframes', name=None, tags=None, recursive=True)

 100% |█████████████| 87306/87306 [5.9s elapsed, 0s remaining, 14.9K samples/s]      


### Kết nối với Pipecone

In [4]:
import pinecone      

pinecone.init(      
	api_key='xxx',      
	environment='gcp-starter'      
)      
# index = pinecone.Index('fiftyone-2023-10-14-14-19-16')

/Users/duyennguyen/anaconda3/lib/python3.10/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


### Tải dữ liệu có sẵn
Trong trường hợp đã train và tính độ tương đồng bạn có thể sử dụng các lệnh sau để tải dữ liệu có sẵn lên.

In [5]:
# # Reload dataset

# import fiftyone as fo

# # The directory containing the dataset to import
# dataset_dir = "/Users/duyennguyen/Downloads/AIC/Batch_01/Pinecone_FiftyOne/pipecone_index_141023"

# # The type of the dataset being imported
# dataset_type = fo.types.FiftyOneDataset  # for example

# # Import the dataset
# dataset = fo.Dataset.from_dir(
#     dataset_dir=dataset_dir,
#     dataset_type=dataset_type,
# )

In [6]:
# dataset.load_brain_view("pinecone_index")

### Kiểm tra dữ liệu tải lên

In [7]:
print(dataset.first())

<Sample: {
    'id': '652bf66cb962c3d1fd378d95',
    'media_type': 'image',
    'filepath': '/Users/duyennguyen/Downloads/AIC/Batch_01/Keyframes/Batch_01/keyframes-1/L07_V001/0001.jpg',
    'tags': [],
    'metadata': None,
}>


### Trích xuất thêm thông tin tên của video và frameid
Thông tin `video` và `frameid` sẽ được lấy từ tên của tập tin keyframe.

In [8]:
for sample in dataset:
    _, sample['video'], sample['frameid'] = sample['filepath'][:-4].rsplit('/', 2)
    sample.save()

### Thêm thông tin kết quả của object detection.

In [9]:
#label_set = set()

for sample in dataset:
    object_path = f"/Users/duyennguyen/Downloads/AIC/Batch_01/objects/{sample['video']}/{sample['frameid']}.json"
    with open(object_path) as jsonfile:
        det_data = json.load(jsonfile)
    detections = []
    for cls, box, score in zip(det_data['detection_class_entities'], det_data['detection_boxes'], det_data['detection_scores']):
        # Convert to [top-left-x, top-left-y, width, height]
        boxf = [float(box[1]), float(box[0]), float(box[3]) - float(box[1]), float(box[2]) - float(box[0])]
        scoref = float(score)

        # Only add objects with confidence > 0.4
        if scoref > 0.4:
            detections.append(
                fo.Detection(
                    label=cls,
                    bounding_box= boxf,
                    confidence=float(score)
                )
            )
    # for detection in detections:
    #     label_set.add(detection.label)
        
    sample["object_faster_rcnn"] = fo.Detections(detections=detections)
    sample.save()
    
# # save label_set as a json file
# with open('label_set.json', 'w') as f:
#     json.dump(list(label_set), f)

### Thêm thông tin CLIP embedding.

In [10]:
all_keyframe = glob('/Users/duyennguyen/Downloads/AIC/Batch_01/Keyframes/*/keyframes-*/*/*.jpg')
video_keyframe_dict = {}
all_video = glob('/Users/duyennguyen/Downloads/AIC/Batch_01/Keyframes/*/keyframes-*/*')
all_video = [v.rsplit('/',1)[-1] for v in all_video]

Tạo dictionary `video_keyframe_dict` với `video_keyframe_dict[video]` thông tin danh sách `keyframe` của `video`

In [11]:
for kf in all_keyframe:
    _, vid, kf = kf[:-4].rsplit('/',2)
    if vid not in video_keyframe_dict.keys():
        video_keyframe_dict[vid] = [kf]
    else:
        video_keyframe_dict[vid].append(kf)

Do thông tin vector CLIP embedding được cung cấp được lưu theo từng video nhằm mục đích tối ưu thời gian đọc dữ liệu. Cần sort lại danh sách `keyframe` của từng `video` để đảm bảo thứ tự đọc đúng với vector embedding được cung cấp.

In [12]:
for k,v in video_keyframe_dict.items():
    video_keyframe_dict[k] = sorted(v)

Tạo dictionary `embedding_dict` với `embedding_dict[video][keyframe]` lưu thông tin vector CLIP embedding của `keyframe` trong `video` tương ứng

In [13]:
embedding_dict = {}
for v in all_video:
    clip_path = f'/Users/duyennguyen/Downloads/AIC/Batch_01/clip-features-vit-b32/{v}.npy'
    a = np.load(clip_path)
    embedding_dict[v] = {}
    for i,k in enumerate(video_keyframe_dict[v]):
        embedding_dict[v][k] = a[i]

Tạo danh sách `clip_embedding` ứng với danh sách `sample` trong `dataset`.

In [14]:
clip_embeddings = []
for sample in dataset:
    clip_embedding = embedding_dict[sample['video']][sample['frameid']]
    clip_embeddings.append(clip_embedding)


### Xoá brain run trong trường hợp xảy ra lỗi brain run đã tồn tại

In [15]:
# # Clean up    
# # Delete run record from FiftyOne
# dataset.delete_brain_run("pinecone_index") # milvus_index, img_sim, tmp

### Tính độ tương đồng và lưu vào Pipecone

In [16]:
pinecone_index = fob.compute_similarity(

    # pinecone
    dataset,
    model="clip-vit-base32-torch",      # store model's name for future use
    embeddings=clip_embeddings,       # precomputed image embeddings
    backend="pinecone",                 # sklearn, qdrant, pinecone, milvus, lancedb
    brain_key="pinecone_index",             # img_sim, tmp, milvus_index, lancedb_index
    environment="gcp-starter",
    api_key="xxx",
    metric="cosine", # dotproduct, cosine, euclidean
    pod_type="starter",
    pods=1,
    demensions=512,   
)

Ignoring unsupported parameters {'demensions'} for <class 'fiftyone.brain.internal.core.pinecone.PineconeSimilarityConfig'>


### Lưu lại dữ liệu để dùng lại

In [17]:
dataset.export(
    './pipecone_index_141023',
    dataset_type=fo.types.FiftyOneDataset
    )

In [18]:
print(fob.brain_config)

{
    "default_similarity_backend": "sklearn",
    "similarity_backends": {
        "lancedb": {
            "config_cls": "fiftyone.brain.internal.core.lancedb.LanceDBSimilarityConfig"
        },
        "milvus": {
            "config_cls": "fiftyone.brain.internal.core.milvus.MilvusSimilarityConfig"
        },
        "pinecone": {
            "config_cls": "fiftyone.brain.internal.core.pinecone.PineconeSimilarityConfig"
        },
        "qdrant": {
            "config_cls": "fiftyone.brain.internal.core.qdrant.QdrantSimilarityConfig"
        },
        "sklearn": {
            "config_cls": "fiftyone.brain.internal.core.sklearn.SklearnSimilarityConfig"
        }
    }
}


### Thực hiện truy vấn

In [19]:
session = fo.launch_app(dataset, port =5152)
session.wait()

Notebook sessions cannot wait


In [20]:
# Query by vector
query = np.random.rand(512)  # matches the dimension of CLIP embeddings
view = dataset.sort_by_similarity(query, k=10, brain_key="pinecone_index")
session.view = view

In [21]:
# Query by sample ID
query = dataset.first().id
view = dataset.sort_by_similarity(query, k=10, brain_key="pinecone_index")
session.view = view

In [22]:
# Query by a list of IDs
query = [dataset.first().id, dataset.last().id]
view = dataset.sort_by_similarity(query, k=10, brain_key="pinecone_index")
session.view = view

In [23]:
# Query by text prompt
query = "a house in the middle of the forest"
view = dataset.sort_by_similarity(query, k=10, brain_key="pinecone_index")
session.view = view 

### Thêm - xoá dữ liệu 

In [24]:
# Delete 10 samples from a dataset
dataset.delete_samples(view)

# Delete the corresponding vectors from the index
pinecone_index.remove_from_index(sample_ids=ids)

# Add 20 samples to a dataset
samples = [fo.Sample(filepath="tmp%d.jpg" % i) for i in range(20)]
sample_ids = dataset.add_samples(samples)

# Add corresponding embeddings to the index
embeddings = np.random.rand(20, 512)
pinecone_index.add_to_index(embeddings, sample_ids)
